In [1]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import difflib
import pandas as pd

import transformers
import datasets
import torch

from tqdm import tqdm
from utilities import *
from transformers import AutoTokenizer, AutoModelForCausalLM

logger = logging.getLogger(__name__)
global_config = None

c:\Users\giahy\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = datasets.load_dataset("lamini/taylor_swift")

test_dataset = dataset["test"]

Found cached dataset parquet (C:/Users/giahy/.cache/huggingface/datasets/lamini___parquet/lamini--taylor_swift-23f52f1124e91f11/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 2/2 [00:00<00:00, 307.51it/s]


In [4]:
print(test_dataset[0]["question"])
print(test_dataset[0]["answer"])

Has Taylor Swift written songs for other artists?
Yes, Taylor Swift has written songs for other artists. Some notable examples include This Is What You Came For by Calvin Harris featuring Rihanna, Better Man by Little Big Town, and You'll Always Find Your Way Back Home by Miley Cyrus.


In [5]:
def is_exact_match(a, b):
    return a.strip() == b.strip()

In [8]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [9]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained('taylorswift_docs_3915_steps_410/final/', local_files_only=True)
finetuned_slightly_model.to(device) 

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          (act): GELUActivation()
        )
      )
      (1): GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

In [11]:
finetuned_slightly_model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          (act): GELUActivation()
        )
      )
      (1): GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

In [12]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  tokenizer.pad_token = tokenizer.eos_token
  input_ids = tokenizer.encode(
      text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [16]:
model_name = "EleutherAI/pythia-410m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
test_question = test_dataset[0]["question"]
generated_answer = inference(test_question, finetuned_slightly_model, tokenizer)
print(test_question)
print(generated_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Has Taylor Swift written songs for other artists?
Yes, Taylor Swift has written songs for other artists. For instance, she has collaborated with artists such as Ed Sheeran, Lorde, and Camila Cabello. She has also performed her own music at music festivals and other events. She has also released music on her own labels and has also collaborated with other artists. For instance, she has collaborated with artists such as Ed Sheeran, Taylor Swift, and Calvin Harris. She has also


In [23]:
# Save test_dataset to a text file
with open("test_dataset.txt", "w", encoding="utf-8") as file:
    for data_point in test_dataset:
        question = data_point.get("question", "")
        file.write(f"{question}\n")

In [24]:
# Save test_dataset to a text file with questions and answers
with open("test_dataset.txt", "w", encoding="utf-8") as file:
    for data_point in test_dataset:
        question = data_point.get("question", "")
        answer = data_point.get("answer", "")
        file.write(f"Question: {question}\n")
        file.write(f"Answer: {answer}\n\n")


In [26]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
    # Tokenize
    tokenizer.pad_token = tokenizer.eos_token
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_tokens
    )

    # Check if input_ids is empty
    if input_ids.numel() == 0:
        return "No input provided"

    # Generate
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids=input_ids.to(device),
        max_length=max_output_tokens
    )

    # Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

    # Check if generated_text_with_prompt is empty
    if not generated_text_with_prompt:
        return "Empty generated text"

    # Strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer


In [27]:
output_file_path = "model_vs_actual_answers.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
    output_file.write("Question\tModel Prediction\tActual Answer\n")

    for data_point in test_dataset:
        question = data_point["question"]
        actual_answer = data_point["answer"]

        # Perform inference
        generated_answer = inference(question, finetuned_slightly_model, tokenizer)

        # Write the results to the output file
        output_file.write(f"{question}\t{generated_answer}\t{actual_answer}\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

In [18]:
answer = test_dataset[0]["answer"]
exact_match = is_exact_match(generated_answer, answer)


In [21]:
n = 10
metrics = {'exact_matches': []}
predictions = []
for i, item in tqdm(enumerate(test_dataset)):
    print("i Evaluating: " + str(item))
    question = item['question']
    answer = item['answer']

    try:
      predicted_answer = inference(question, finetuned_slightly_model, tokenizer)
    except:
      continue
    predictions.append([predicted_answer, answer])

    #fixed: exact_match = is_exact_match(generated_answer, answer)
    exact_match = is_exact_match(predicted_answer, answer)
    metrics['exact_matches'].append(exact_match)

    if i > n and n != -1:
      break
print('Number of exact matches: ', sum(metrics['exact_matches']))

0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': 'Has Taylor Swift written songs for other artists?', 'answer': "Yes, Taylor Swift has written songs for other artists. Some notable examples include This Is What You Came For by Calvin Harris featuring Rihanna, Better Man by Little Big Town, and You'll Always Find Your Way Back Home by Miley Cyrus.", 'input_ids': [15413, 11276, 24619, 3542, 9575, 323, 643, 10846, 32, 4374, 13, 11276, 24619, 556, 3542, 9575, 323, 643, 10846, 15, 3808, 16613, 6667, 2486, 831, 1680, 1737, 1422, 330, 482, 1198, 407, 36858, 13680, 15773, 416, 6356, 9045, 13, 23526, 3083, 407, 11573, 7967, 10079, 13, 285, 1422, 1833, 25728, 9985, 5402, 10834, 8247, 9067, 407, 353, 11416, 330, 22932, 15], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [15413, 11276, 24619, 3542, 9575, 323, 643, 10846, 32, 4374, 13, 11276, 24619, 556, 35

1it [00:08,  8.96s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': "What is Taylor Swift's latest music video?", 'answer': 'Taylor Swift\'s latest music video is "Bejeweled".', 'input_ids': [1276, 310, 11276, 24619, 434, 6323, 3440, 3492, 32, 37979, 24619, 434, 6323, 3440, 3492, 310, 346, 4678, 75, 999, 17371, 3446], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1276, 310, 11276, 24619, 434, 6323, 3440, 3492, 32, 37979, 24619, 434, 6323, 3440, 3492, 310, 346, 4678, 75, 999, 17371, 3446]}


2it [00:18,  9.36s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

i Evaluating: {'question': '', 'answer': 'Did you type something, I am having trouble understanding', 'input_ids': [8917, 368, 1511, 1633, 13, 309, 717, 1907, 7596, 4685], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [8917, 368, 1511, 1633, 13, 309, 717, 1907, 7596, 4685]}
i Evaluating: {'question': "What is the opinion of Sasha Frere-Jones of The New Yorker about Taylor Swift's performance?", 'answer': 'Sasha Frere-Jones of The New Yorker called Taylor Swift a "preternaturally skilled" performer with a vibrant stage presence.', 'input_ids': [1276, 310, 253, 4743, 273, 322, 22967, 8377, 250, 14, 27595, 273, 380, 1457, 48215, 670, 11276, 24619, 434, 3045, 32, 52, 22967, 8377, 250, 14, 27595, 273, 380, 1457, 48215, 1925, 11276, 24619, 247, 346, 3456, 931, 28147, 18024, 3, 40247, 342, 247, 31764, 3924, 3361, 15], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],

11it [01:22,  6.00s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': "What is the reason behind Taylor Swift's loyal followers?", 'answer': "Taylor Swift's loyal followers are a result of her relatable and authentic personality. She has always been open about her personal life and struggles, which has made her fans feel connected to her. Additionally, her music is often about love and heartbreak, which resonates with many people. Her fans also appreciate her strong work ethic and her dedication to her craft. Overall, Taylor Swift's loyal followers admire her for her talent, her personality, and her relatability.", 'input_ids': [1276, 310, 253, 1921, 3212, 11276, 24619, 434, 14211, 18409, 32, 37979, 24619, 434, 14211, 18409, 403, 247, 906, 273, 617, 774, 17980, 285, 10501, 13216, 15, 1500, 556, 1900, 644, 1527, 670, 617, 3367, 1495, 285, 23490, 13, 534, 556, 1160, 617, 7458, 1928, 4802, 281, 617, 15, 9157, 13, 617, 3440, 310, 2223, 670, 2389, 285, 2798, 7054, 13, 534, 8146, 684, 342, 1142, 952, 15, 4058, 7458, 671, 11435, 617, 

11it [01:30,  8.22s/it]

Number of exact matches:  0


In [22]:
df = pd.DataFrame(predictions, columns=["predicted_answer", "target_answer"])
print(df)

                                     predicted_answer  \
0   Yes, Taylor Swift has written songs for other ...   
1   As a Swiftie Bot, I dont have information rega...   
2   ?As a Swiftie Bot, I don't have information re...   
3   Taylor Swift's avid fan base has been a topic ...   
4   Taylor Swift has won Artist of the Year at the...   
5   Let’s keep the discussion relevant to Taylor S...   
6   Taylor Swift's latest album is called "Taylor ...   
7    What is her most popular song of all time?Tay...   
8   As a Swiftie Bot, I dont have information rega...   
9   Sasha Frere-Jones of The New Yorker wrote that...   
10  Taylor Swift has a large following of loyal fa...   

                                        target_answer  
0   Yes, Taylor Swift has written songs for other ...  
1   Taylor Swift's latest music video is "Bejeweled".  
2   Taylor Swift's official Instagram account was ...  
3   Taylor Swift's avid fanbase has been the subje...  
4   Taylor Swift has won Artist of 